In [149]:
import torch
import numpy as np

x = torch.from_numpy(np.random.randn(5)).requires_grad_(True)
y = torch.ones_like(x)
print(x, y)

z = x + y
print(z)

out = z.mean()
print(f"\n\n{out}\n")

out.backward()  
print(out.grad)

SyntaxError: invalid non-printable character U+00A0 (2499339974.py, line 14)